In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


In [2]:
learning_rate = 0.00001
epochs = 10
batch_size = 128

test_valid_size = 256

n_classes = 10
dropout = 0.75

In [3]:
weights = {
    'wc1': tf.Variable(tf.random_normal([5,5,1,32])),
    'wc2': tf.Variable(tf.random_normal([5,5,32,64])),
    'wd1': tf.Variable(tf.random_normal([7 * 7 * 64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [4]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(
        x, 
        W, 
        strides=[1, strides, strides, 1], 
        padding="SAME"
    )
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1,k,k,1],
        strides=[1,k,k,1],
        padding="SAME"
    )


In [ ]:
def conv_net(x, weights, biases, dropout):
    conv1 = conv2d(
        x, 
        weights['wc1'], 
        biases['bc1']
    )
    conv1 = maxpool2d(conv1, k=2)
    
    conv2 = conv2d(
        conv1, 
        weights['wc2'], 
        biases['bc2']
    )
    conv2 = maxpool2d(conv2, k=2)
    
    fc1 = tf.reshape(
        conv2, 
        [
            -1, 
            weights['wd1'].get_shape().as_list()[0]
        ]
    )
    fc1 = tf.add(
        tf.matmul(fc1, weights['wd1']),
        biases['bd1']
    )
    fc1 = tf.nn.relu(fc1)
    fc = tf.nn.dropout(fc1, dropout)
    
    out = tf.add(
        tf.matmul(fc1, weights['out']),
        biases['out']
    )
    return out